# Assignment 4

In [1]:
import numpy as np   # Do not modify these imports
import pandas as pd
import math

### Part A

A bond with a face value of ```FV``` and a coupon rate of ```cpn_rate``` will mature `yrs` years and `mts` months from today. The bond pays coupons semi-annually, with the next coupon to be paid 3 months from today. Suppose the yield to maturity of this bond is ```r``` (APR, compounded semi-annually). The parameters `yrs` and `mts` will be non-negative integers, with `mts < 12`.

Write the function ```clean_price```, which returns the clean price of the bond today.

In [2]:
def clean_price(FV, cpn_rate, r, yrs, mts):
    n = math.ceil(yrs * 2 + mts / 6)
    ytm = r * 1/2
    cpn = cpn_rate * FV * 1/2
    acc_interest = cpn * 1/2
    dirty_price = ((1 + ytm) ** (1/2)) * ((cpn/ytm) * (1 - 1/(1 + ytm) ** n) + FV/(1 + ytm) ** n)
    clean_price = dirty_price - acc_interest
    return clean_price

In [3]:
clean_price(1000, 0.06, 0.058, 3, 3)

1005.7402358063853

Suppose that a zero-coupon bond with a face value of ```FV``` costs ```C1``` and will mature 2 years from today. An otherwise identical bond with maturity 3 years from today is selling for ```C2```. Write the function ```forward_rate```, which calculates the forward rate available today for a period of one year starting 2 years from today.

In [4]:
def forward_rate(FV, C1, C2):
    return (FV/C2) / (FV/C1) - 1

In [5]:
forward_rate(1000, 902.21, 852.34)

0.058509514982284294

### Part B

We wish to study the relation between bond rating and bond yield. 

We will first convert S&P ratings to numerical numbers as follows. Assign 1 to the lowest possible rating, and assign 2 to the second lowest rating, etc. We need to do the same for Moody’s ratings. Please use the conversion functions (```SP_convert```,```moody_convert```)  that are already provided below.

Note that for all following functions, we will only be considering bonds with valid ratings. In other words, if any of these conversion functions returns -1 (signifying an invalid rating), we will not consider it in our calculations. Such bonds should be removed from the data.

In [6]:
# Converts an S&P rating to a number described by the system above.
# Converts all invalid ratings to -1.
def SP_convert(rating):
    if rating == "D":
        return 0
    if rating == "C":
        return 1
    if rating == "CC":
        return 2
    if rating == "CCC-":
        return 3
    if rating == "CCC":
        return 4
    if rating == "CCC+":
        return 5
    if rating == "B-":
        return 6
    if rating == "B":
        return 7
    if rating == "B+":
        return 8
    if rating == "BB-":
        return 9
    if rating == "BB":
        return 10
    if rating == "BB+":
        return 11
    if rating == "BBB-":
        return 12
    if rating == "BBB":
        return 13
    if rating == "BBB+":
        return 14
    if rating == "A-":
        return 15
    if rating == "A":
        return 16
    if rating == "A+":
        return 17
    if rating == "AA-":
        return 18
    if rating == "AA":
        return 19
    if rating == "AA+":
        return 20
    if rating == "AAA":
        return 21
    else:
        return -1

# Converts Moody's ratings to a number described by the system above.
# Converts all invalid ratings to -1.
def moody_convert(rating):
    if rating == "C":
        return 0
    if rating == "Ca":
        return 1
    if rating == "Caa3":
        return 2
    if rating == "Caa2":
        return 3
    if rating == "Caa1":
        return 4
    if rating == "B3":
        return 5
    if rating == "B2":
        return 6
    if rating == "B1":
        return 7
    if rating == "Ba3":
        return 8
    if rating == "Ba2":
        return 9
    if rating == "Ba1":
        return 10
    if rating == "Baa3":
        return 11
    if rating == "Baa2":
        return 12
    if rating == "Baa1":
        return 13
    if rating == "A3":
        return 14
    if rating == "A2":
        return 15
    if rating == "A1":
        return 16
    if rating == "Aa3":
        return 17
    if rating == "Aa2":
        return 18
    if rating == "Aa1":
        return 19
    if rating == "Aaa":
        return 20
    else:
        return -1
    

# The next two functions are variants of the above that can be run on columns.

def SP_convert_column(column):  
    tmp = np.vectorize(SP_convert)
    return pd.Series(tmp(column))

def moody_convert_column(column):
    tmp = np.vectorize(moody_convert)
    return pd.Series(tmp(column))

Write the function ```SP_investment_grade```, which  which takes a path to the ratings csv file (similar to previous assignments), and returns the percentage of bonds with investment grades according to S&P. 

Please see Table 6.4 in the textbook to determine which ratings correspond to investment grade. 

For this analysis, two files are provided (which are originally supplised by the data vendor, FISD). Please see the attached PDF for more data descriptions. More specifically, please see Pg31-32: “Rating” for "mod4_ratings.csv" file and pg 19-23: “Issue” for "mod4_bonds.csv" file. 

In [7]:
def SP_investment_grade(path):
    df = pd.read_csv(path) 

    df = df[df["RATING_TYPE"] == "SPR"]
    sp_ratings = SP_convert_column(df["RATING"])
    valid_bond_ratings = sp_ratings[sp_ratings != -1]
    investment_grade_bond_ratings = valid_bond_ratings[valid_bond_ratings >= 12]
    return len(investment_grade_bond_ratings) / len(valid_bond_ratings)


Write the function ```moody_investment_grade```, which does the same thing as ```SP_investment_grade```, but according to Moody's instead.

In [8]:
def moody_investment_grade(path):
    df = pd.read_csv(path) 

    df = df[df["RATING_TYPE"] == "MR"]
    moody_ratings = moody_convert_column(df["RATING"])
    valid_bond_ratings = moody_ratings[moody_ratings != -1]
    investment_grade_bond_ratings = valid_bond_ratings[valid_bond_ratings >= 11]
    return len(investment_grade_bond_ratings) / len(valid_bond_ratings)


Using the conversion system described above, write the function ```SP_correlation```, which finds the correlation between between S&P rating and bond yield. This function will take two arguments, ```bonds_path```, which is a file path to the bonds csv, and ```ratings_path```, which is a file path to the ratings csv. 

In the same manner, write the function ```moody_correlation```, which does the same thing as ```SP_correlation```, but using the Moody Rating instead.

"OFFERING_YIELD" column sometimes contains unreasonable numbers. And that is most likely due to data glitch. Please address the data glitch problem by setting "OFFERING_YIELD" to 30 if it is larger than 30. 

In this question, you may find it helpful to merge two DataFrames. You can achieve this with the line

`pd.merge(df_1, df_2, how = 'inner', on = ["Column Name"])`

This merges the two dataframes `df_1` and `df_2` based on matching values in the column with name "Column Name", returning the value of the merged DataFrame.

In [9]:
def SP_correlation(bonds_path, ratings_path):
    bonds_df = pd.read_csv(bonds_path)
    ratings_df = pd.read_csv(ratings_path) # Do not change the first 3 lines of this function.
    
    df = pd.merge(bonds_df, ratings_df, how = 'inner', on = ["ISSUE_ID"])
    df = df[df["RATING_TYPE"] == "SPR"]
    df["S&P_RATING"] = SP_convert_column(df["RATING"]).values
    df = df[df["S&P_RATING"] != -1]
    df = df.dropna(subset=["OFFERING_YIELD"])
    df.loc[df['OFFERING_YIELD'] > 30, 'OFFERING_YIELD'] = 30
    return df['OFFERING_YIELD'].corr(df["S&P_RATING"])

In [10]:
def moody(bonds_path, ratings_path):
    bonds_df = pd.read_csv(bonds_path)
    ratings_df = pd.read_csv(ratings_path) # Do not change the first 3 lines of this function.
    
    df = pd.merge(bonds_df, ratings_df, how = 'inner', on = ["ISSUE_ID"])
    df = df[df["RATING_TYPE"] == "MR"]
    df["MOODY_RATING"] = moody_convert_column(df["RATING"]).values
    df = df[df["MOODY_RATING"] != -1]
    df = df.dropna(subset=["OFFERING_YIELD"])
    df.loc[df['OFFERING_YIELD'] > 30, 'OFFERING_YIELD'] = 30

    return df['OFFERING_YIELD'].corr(df["MOODY_RATING"])